In [1]:
import scanpy as sc
import scvi

/software/cellgen/team278/bc8/envs/scvi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.read('/lustre/scratch126/cellgen/team292/ha10/data/Fetal_Lung/adata_fetal_lung_training_noccgene.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 65564 × 4980
    obs: 'batch', 'stage', 'dissection', 'chemistry', 'percent_mito', 'n_counts', 'n_genes', 'doublet_scores', 'bh_pval', 'leiden', 'phase', 'S_score', 'G2M_score', 'new_celltype', 'big_cluster', 'donor', 'gender', 'new_celltype-code', 'status_summary', 'productive_summary', 'isotype_summary', 'receptor_type', 'receptor_subtype', 'broad_celltype', 'cycling', 'study', 'stage_coarse'
    var: 'gene_ids', 'n_cells', 'highly_variable5891STDY8062349', 'highly_variable5891STDY8062350', 'highly_variable5891STDY8062351', 'highly_variable5891STDY8062352', 'highly_variable5891STDY8062353', 'highly_variable5891STDY8062354', 'highly_variable5891STDY8062355', 'highly_variable5891STDY8062356', 'highly_variableWSSS8012016', 'highly_variableWSSS8011222', 'highly_variableWSSS_F_LNG8713176', 'highly_variableWSSS_F_LNG8713177', 'highly_variableWSSS_F_LNG8713178', 'highly_variableWSSS_F_LNG8713179', 'highly_variableWSSS_F_LNG8713180', 'highly_variableWSSS_

In [4]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="donor_id")

In [9]:
model = scvi.model.SCVI(adata, n_layers=2, n_latent=25, gene_likelihood="nb")

In [10]:
model.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 28/28: 100%|██████████| 28/28 [04:41<00:00,  9.99s/it, v_num=1, train_loss_step=667, train_loss_epoch=662]

`Trainer.fit` stopped: `max_epochs=28` reached.


Epoch 28/28: 100%|██████████| 28/28 [04:41<00:00, 10.05s/it, v_num=1, train_loss_step=667, train_loss_epoch=662]


In [11]:
SCVI_LATENT_KEY = "X_scVI"
adata.obsm[SCVI_LATENT_KEY] = model.get_latent_representation()

In [12]:
sc.pp.neighbors(adata, use_rep=SCVI_LATENT_KEY)
sc.tl.leiden(adata)

/tmp/ipykernel_274428/2529970008.py:2: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata)


In [13]:
sc.tl.umap(adata)

In [14]:
SCVI_MDE_KEY = "X_scVI_MDE"
adata.obsm[SCVI_MDE_KEY] = scvi.model.utils.mde(adata.obsm[SCVI_LATENT_KEY])

INFO     Using cuda:0 for `pymde.preserve_neighbors`.                                                              


In [16]:
adata.write('/lustre/scratch127/cellgen/cellgeni/tickets/tic-3135/adata_scvi.h5ad', compression='gzip')